In [26]:
import numpy as np
import math
from IPython.core.debugger import set_trace

lowerBound, upperBound = -30, 30
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)

def f(x):
    #Sphere
    #Sum = 0
    #for i in range(0,N):
    #    Sum = Sum + (x[i])**2

    #Rosenbrock
    #Sum = 0
    #for i in range(0, N-1):
    #    Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley
    Sum1, Sum2 = 0, 0
    for i in range(0,N):
        Sum1 = Sum1 + (x[i])**2
        Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum

l = [lowerBound for i in range(10)]
u = [upperBound for i in range(10)]

In [27]:
def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr

In [28]:
def Delta(x):
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum

In [29]:
def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False

In [30]:
def findExtremes(s):
    Ds = np.empty([N+1])
    fs = np.empty([N+1])
    
    for i in range(0, N+1):
        Ds[i] = Delta(s[i])
        fs[i] = f(s[i])
    
    arg_b = np.argmin(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_b] and i != arg_b:
            if(fs[i] < fs[arg_b]):
                arg_b = i;
                
    arg_w = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_w] and i != arg_w:
            if(fs[i] > fs[arg_w]):
                arg_w = i;
    
    #mask the worst to find the second worst
    Ds[arg_w] = 0
    
    arg_sw = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_sw] and i != arg_w:
            if(fs[i] > fs[arg_sw]):
                arg_sw = i;
    
    return arg_b,arg_w,arg_sw

In [31]:
def centroid(s,argWorst):
    return np.mean(np.delete(s, argWorst), axis = 0)

In [32]:
def SimplexLocalSearch(x,Epsilon,Lambda,M,Alpha=1,Gamma=2,Beta=-0.5):
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N);
    k = 0
    
    
    argBest, argWorst, argSecondWorst = findExtremes(s)
    sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
    #set_trace()
    while abs(f(sBest) - f(sWorst)) > Epsilon and k < M:
        #set_trace()
        if(k%1000 == 0):print(f(sBest),'\n')
        r = (1+Alpha)*centroid(s,argWorst) - Alpha*sWorst
        if(better(sBest,r) and better(r,sSecondWorst)):
            s[argWorst] = r
        elif(better(r,sBest)):
            e = (1+Gamma)*centroid(s,argWorst)-Gamma*sWorst
            if(better(e,r)):
                s[argWorst] = e
            else:
                s[argWorst] = r
        else:
            c = (1+Beta)*centroid(s,argWorst)-Beta*sWorst
            if(better(c,sWorst)):
                s[argWorst] = c
            else:
                for j in range(0,N+1):
                    s[j] = (s[j] + sBest)/2

        #set_trace()
        argBest, argWorst, argSecondWorst = findExtremes(s)
        sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
        k = k + 1
    print(f(sBest))
    return sBest

In [33]:
def Simplex(x,Epsilon, Lambda, M):
    '''read problem data'''
    xBest = SimplexLocalSearch(x,Epsilon, Lambda,M);
    return xBest

In [34]:
Lambda = 1
M = 100000
Epsilon = 0
print(Simplex(Epsilon,Lambda,M))

TypeError: Simplex() missing 1 required positional argument: 'M'

In [35]:
# simulated annealing search of a one-dimensional objective function
from numpy import asarray
from numpy import exp
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed
import math

N = 10
# objective function
def objective(x):
    Sum1 = 0
    Sum2 = 0
    for i in range(0,N):
        Sum1 = Sum1 + (x[i])**2
        Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    Sum = -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum
    
# simulated annealing algorithm
def simulated_annealing(objective, bounds, n_iterations, step_size, temp):
    # generate an initial point
    best = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
    # evaluate the initial point
    best_eval = objective(best)
    # current working solution
    curr, curr_eval = best, best_eval
    # run the algorithm
    for i in range(n_iterations):
        Lambda = 1
        M = 100000
        Epsilon = 0
        # take a step
        candidate = Simplex(curr,Epsilon,Lambda,M)
        # evaluate candidate point
        candidate_eval = objective(candidate)
        # check for new best solution
        if candidate_eval < best_eval:
            # store new best point
            best, best_eval = candidate, candidate_eval
            # report progress
            print('>%d f(%s) = %.5f' % (i, best, best_eval))
        # difference between candidate and current point evaluation
        diff = candidate_eval - curr_eval
        # calculate temperature for current epoch
        t = temp / float(i + 1)
        # calculate metropolis acceptance criterion
        metropolis = exp(-diff / t)
        # check if we should keep the new point
        if diff < 0 or rand() < metropolis:
            # store the new current point
            curr, curr_eval = candidate, candidate_eval
    return [best, best_eval]

# seed the pseudorandom number generator
seed(1)
# define range for input
bounds = asarray([[-30.0, 30.0],[-30.0, 30.0],[-30.0, 30.0],[-30.0, 30.0],[-30.0, 30.0],[-30.0, 30.0],[-30.0, 30.0],[-30.0, 30.0],[-30.0, 30.0],[-30.0, 30.0]])
# define the total iterations
n_iterations = 1000
# define the maximum step size
step_size = 0.1
# initial temperature
temp = 10
# perform the simulated annealing search
best, score = simulated_annealing(objective, bounds, n_iterations, step_size, temp)
print('Done!')
print('f(%s) = %f' % (best, score))

20.729779283862474 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 

9.660754435231537 



KeyboardInterrupt: 